## 2020 MagIC Workshop PmagPy notebook demo



- Go to the jupyter-hub website to run this online, or put this notebook in some working directory.   

- Click on the cell below and then click on 'Run' from the menu above to import the desired functionality

In [ ]:
# Import PmagPy modules
import pmagpy.pmag as pmag
import pmagpy.pmagplotlib as pmagplotlib
import pmagpy.ipmag as ipmag
import pmagpy.contribution_builder as cb
from pmagpy import convert_2_magic as convert

# Import plotting modules
has_cartopy, Cartopy = pmag.import_cartopy() # import mapping module, if it is available
import matplotlib.pyplot as plt # our plotting buddy
# This allows you to make matplotlib plots inside the notebook.  
%matplotlib inline 

# Import more useful modules
import numpy as np # the fabulous NumPy package
import pandas as pd # and  Pandas for data wrangling
import os # some useful operating system functions
from importlib import reload # for reloading module if they get changed after initial import
import imageio # for making animations
import wget

# make the example directories if they don't already exist
!mkdir -p MagIC_example_1
!mkdir -p MagIC_example_2
!mkdir -p MagIC_example_3

## Overview of demonstration

This notebook has three exercises that demonstrate various aspects of the PmagPy software.  
- Exercise 1 looks at a typical "directional" data set and shows how to make useful plots like the equal area projection, maps of VGPs and basemaps of site locations.  
- Exercise 2 shows how to get geomagnetic vectors from IGRF-like tables and several ways of looking at the data through time and space. 
- Exercise 3 considers directional (polariy) and anisotropy data as a function of depth in an IODP core.  

When you have worked through these examples, check out the other PmagPy notebooks in the software distribution. 

## Exercise 1

- Hunt around the earthref.org/MagIC/search page for a data set you would like to look at. We will use the data of Behar et al., 2019,  DOI: 10.1029/2019GC008479 for this exercise. 
- Download the associated datafile, magic_contribution_16676.txt,  using _wget()_ and move it to the working directory, MagIC_example_1
- unpack it with _ipmag.download_magic()_
- Use PmagPy functions to make the following plots:
    - use _ipmag.eqarea_magic()_ to make an equal area plot
    - use _ipmag.vgpmap_magic()_ to make a map of VGPs
    - use _ipmag.reversal_test_bootstrap()_ for a bootstrap reversals test
    - use _pmagplotlib.plot_map()_ to make a site map

In [ ]:
dir_path='MagIC_example_1' # set the path to the correct working directory
# First get the file from MagIC
wget.download('https://earthref.org/MagIC/download/16676/magic_contribution_16676.txt')
# and move it into the working directory:
try:
    !mv magic_contribution_16676.txt MagIC_example_1
except:
    print ('Please create a directory on your PC and move this data file into it.\n  Then change dir_path to match')

To understand what a particular PmagPy function expects as input and delivers, use the Python _help_ function

In [ ]:
help(ipmag.download_magic)

In [ ]:
# unpack the downloaded file with the ipmag function download_magic()
ipmag.download_magic('magic_contribution_16676.txt',dir_path=dir_path,print_progress=False)

### Equal area net example
- use ipmag.eqarea_magic()

In [ ]:
# first get help on how to use it:
help(ipmag.eqarea_magic)

In [ ]:
# now we do it for real:
ipmag.eqarea_magic(dir_path=dir_path,save_plots=False)

### Map of VGPs
- use ipmag.vgpmap_magic() to plot the VGPs from the same data

In [ ]:
# get help message for vgpmap_magic
help(ipmag.vgpmap_magic)

In [ ]:
dir_path='MagIC_example_1' # set the path to your working directory
ipmag.vgpmap_magic(dir_path=dir_path,size=50,flip=True,save_plots=False,lat_0=60,rsym='b^',rsize=50)

### Bootstrap reversals test
- use ipmag.reversal_test_bootstrap() to do the reversals test

In [ ]:
help(ipmag.reversal_test_bootstrap)

In [ ]:
# read in the data into a Pandas DataFrame
dir_path='MagIC_example_1' # set the path to your working directory
sites_df=pd.read_csv(dir_path+'/sites.txt',sep='\t',header=1)
# pick out the declinations and inclinations
decs=sites_df.dir_dec.values
incs=sites_df.dir_inc.values
# call the function
ipmag.reversal_test_bootstrap(dec=decs,inc=incs,plot_stereo=False)

### Make a site map
- use pmagplotlib.plot_map() to make a site map

In [ ]:
help(pmagplotlib.plot_map)

NB: the most recent PmagPy version fixes the scale issue - but it is SLOW at high resolution... so set Opts\['res'\] to 'c' for crude for a quick look.  if you want to be dazzled - set it to 'h' but be prepared to wait for a while...  'i' for intermediate is probably good enough for most purposes (50m resolution)

In [ ]:
dir_path='MagIC_example_1' # set the path to your working directory
# read in the data file:
site_df=pd.read_csv(dir_path+'/sites.txt',sep='\t',header=1)
# pick out the longitudes and latitudes
lons=site_df['lon'].values
lats=site_df['lat'].values
# set some options
Opts={}
Opts['sym']='r*' # sets the symbol to white dots
Opts['symsize']=100 # sets symbol size to 3 pts
Opts['proj']='lcc' # Lambert Conformal projection
Opts['pltgrid']=True
Opts['lat_0']=33
Opts['lon_0']=35
Opts['latmin']=29
Opts['latmax']=35
Opts['lonmin']=32
Opts['lonmax']=37
Opts['gridspace']=1
Opts['details']={}
Opts['details']['coasts']=True
Opts['details']['ocean']=True
Opts['details']['countries']=True
Opts['global']=False
Opts['res']='i'
plt.figure(1,(10,10)) # optional - make a map

pmagplotlib.plot_map(1, lats, lons, Opts)


## Exercise 2: 
- use _pmag.pinc()_ to calculate the GAD inclination for a  particular latitude (e.g., 33)
- use _ipmag.igrf()_ and _ipmag.igrf_print()_ to get values of the field for a specific place (e.g., San Diego at lon=-117,lat=33,alt=0) and date (2019)
- make a plot of declination, inclination, B for a specific place and range of dates
- use _pmag.do_mag_map()_ and _pmagplotlib.plot_mag_map()_ to make a map of the field at a specific date (2019)
- make a movie of the field for the last 1000 years using the cals10k.2 model of Constable et al. (2016)

In [ ]:
help(pmag.pinc)

In [ ]:
gad_inc=pmag.pinc(33)
print(gad_inc)

In [ ]:
help(ipmag.igrf)

In [ ]:
date,lat,lon,alt=2019.9,33,-117,0
local=ipmag.igrf([date, alt, lat, lon])
ipmag.igrf_print(local)

In [ ]:
mod='cals10k.2'
dates=range(-8000,2050,50)
local_vectors=[]
for d in dates:
    local=ipmag.igrf([d, alt, lat, lon],mod=mod)
    local_vectors.append([d,local[0],local[1],local[2]])
df=pd.DataFrame(local_vectors,columns=['Date','Dec','Inc','B_nT'])
df['B_uT']=df['B_nT']*1e-3
df.loc[df['Dec']>180,'Dec']=df['Dec']-360. 
fig=plt.figure(1,(8,8))
ax1=fig.add_subplot(311)
ax2=fig.add_subplot(312)
ax3=fig.add_subplot(313)
ax1.plot(df['Date'],df['Dec'])
ax1.axhline(0,color='black',linestyle='dotted')
ax2.plot(df['Date'],df['Inc'])
ax2.axhline(gad_inc,color='black',linestyle='dotted')
ax3.plot(df['Date'],df['B_uT'])

In [ ]:
help(pmag.do_mag_map)

In [ ]:
help(pmagplotlib.plot_mag_map)

In [ ]:
# define some useful parameters
date,mod,lon_0,alt,ghfile=2019,'cals10k.2',0,0,"" # only date is required
Ds,Is,Bs,Brs,lons,lats=pmag.do_mag_map(date,mod=mod,lon_0=lon_0,alt=alt,file=ghfile)

In [ ]:
help(pmagplotlib.plot_mag_map)

In [ ]:
cmap='jet' # nice color map for contourf
pmagplotlib.plot_mag_map(1,Bs,lons,lats,'B',cmap=cmap,date=date,proj='Mollweide',contours=False) # plot the field strength
pmagplotlib.plot_mag_map(2,Is,lons,lats,'I',cmap=cmap,date=date,proj='Mollweide',contours=True)# plot the inclination
pmagplotlib.plot_mag_map(3,Ds,lons,lats,'D',cmap=cmap,date=date,contours=True);# plot the declination    


In [ ]:
dir_path='MagIC_example_2'
!rm -f MagIC_example_2/*.png # remove any old plots
mod,lon_0,alt,ghfile='cals10k.2',0,0,"" #
cmap,title='jet','Intensity' # nice color map for contourf
fignum=1
dates=range(-1000,2100,100) # make maps for these years.  
lon_0=0 # center the maps at the Greenwich meridian
element='B' # let's do field strength
for date in dates:  # step through the loop
    print ('working on: ',date)
    Ds,Is,Bs,Brs,lons,lats=pmag.do_mag_map(date,mod=mod,lon_0=lon_0,alt=alt,file=ghfile)
    pmagplotlib.plot_mag_map(fignum,Bs,lons,lats,'B',cmap=cmap,date=date,proj='Mollweide',
                             min=15,max=100,contours=False) # plot the field strength
    plt.savefig(dir_path+'/' +title.strip()+'_'+str(date)+'.png') # saves the  figure. to a folder
    fignum+=1

In [ ]:
from IPython.display import Image
Image(filename="MagIC_example_2/Bmovie.gif")

In [ ]:
filenames=os.listdir('MagIC_example_2/') # listing of the directory
images = [] # make a container to put the image files in
for file in filenames: # step through all the maps
    if '.png' in file: # skip some of the nasty hidden files
        filename='MagIC_example_2/'+file # make filename from the folder name and the file name
        images.append(imageio.imread(filename)) # read it in and stuff in the container
imageio.mimsave('MagIC_example_2/Bmovie.gif', images) # save to an animated gif.  

Now open MagIC_example_2/Bmovie.gif in your browser. 

## Exercise 3
- download the data from Tauxe et al. (2015; DOI: 0.1016/J.EPSL.2014.12.034) using the wget.download command as in Exercise 1.
- Move the downloaded data file to a directory called 'MagIC\_example\_3'

- Unpack it with ipmag.download_magic()
- make a figure with these elements for the interval 40 m to 160 m:
    - magstrat time scale plot from 2 to 7 Ma
    - inclinations (dir\_inc) from the 20mT step in the measurements table  against composite_depth as blue dots
    - inclinations (dir\_inc) from the specimens table against composite depth as red triangles. 
    - put on dotted lines for the GAD inclination
- use ipmag.ani_depthplot to plot the anisotropy data against depth in the Hole.  

### Download and unpack the data

- set the first if statement to True to download, move and unpack the data set from MagIC

In [ ]:
dir_path='MagIC_example_3' # set the path to your working directory
depth_min, depth_max= 40, 160 # set the core depth bounds as required
# First get the file from MagIC into your working directory:
if False:
    wget.download('https://earthref.org/MagIC/download/16749/magic_contribution_16749.txt')
    # move to the working directory
    !mv magic_contribution_16749.txt MagIC_example_3
    # unpack it:
    ipmag.download_magic(infile='magic_contribution_16749.txt',dir_path=dir_path)
# if you downloaded the excel file instead, then do this:
if False:
    dir_path='MagIC_example_3'
    ipmag.download_magic(dir_path=dir_path,infile='REPLACE_WITH_NAME_OF_FILE.xlsx',excel=True)

## Magstrat figure
- read in the data file as a Pandas DataFrame with pd.read_csv().  
    - All MagIC .txt files are tab delimited.  This is indicated with a sep='\t' keywork.  
    - The column headers in the second row, hence (because Python counts from zero), header=1
- the depth of a particular specimen/site in MagIC is stored in the sites.txt table.  You will have to merge the data from that table into the specimens/measurements tables.  To do that you need to do a few things:
    - you need a common key.  Because the specimen/sample/site names are the same for an IODP record, make a column in the specimen/measurements dataframes labled 'site' that is the same as the specimen.  
    - merge the two dataframes (sites and specimens/measurements) with pd.merge()

In [ ]:
# read in the required data tables:
meas_df=pd.read_csv(dir_path+'/measurements.txt',sep='\t',header=1)
site_df=pd.read_csv(dir_path+'/sites.txt',sep='\t',header=1)
spec_df=pd.read_csv(dir_path+'/specimens.txt',sep='\t',header=1)
ages_df=pd.read_csv(dir_path+'/ages.txt',sep='\t',header=1)
# filter the ages table for method codes that indicate paleomagnetic reversals:
ages_df=ages_df[ages_df['method_codes'].str.contains('PMAG')]
# filter the measurements for the 20 mT (.02 T) step
meas_df.dropna(subset=['treat_ac_field'],inplace=True)
meas_20mT=meas_df[meas_df['treat_ac_field']==0.02] 
# make the site key in the measurements and specimens dataframes
meas_20mT['site']=meas_20mT['specimen']
spec_df['site']=spec_df['specimen']
# we only want the core depth out of the sites dataframe, so we can pare it down like this:
depth_df=site_df[['site','core_depth']]
# merge the specimen, depth dataframes
spec_df=pd.merge(spec_df,depth_df,on='site')
# merge the measurements, depth dataframes
meas_20mT=pd.merge(meas_20mT,depth_df,on='site')
# filter for the desired depth range: 
spec_df=spec_df[(spec_df['core_depth']>depth_min)&(spec_df['core_depth']<depth_max)]
meas_20mT=meas_20mT[(meas_20mT['core_depth']>depth_min)&(meas_20mT['core_depth']<depth_max)]
# note that the age table has only height (not depth), so these numbers are the opposite
ages_df=ages_df[(ages_df['tiepoint_height']<-depth_min)&(ages_df['tiepoint_height']>-depth_max)]
# get the site latitude (there is only one)
lat=site_df['lat'].unique()[0]



In [ ]:
fig=plt.figure(1,(9,12)) # make the figure
ax1=fig.add_subplot(131) # make the first of three subplots
pmagplotlib.plot_ts(ax1,2,7,timescale='gts12') # plot on the time scale
ax2=fig.add_subplot(132) # make the second of three subplots
plt.plot(meas_20mT.dir_inc,meas_20mT.core_depth,'bo',markeredgecolor='black',alpha=.5)
plt.plot(spec_df.dir_inc,spec_df.core_depth,'r^',markeredgecolor='black')

plt.ylim(depth_max,depth_min)
# calculate the geocentric axial dipole field for the site latitude
gad=pmag.pinc(lat) # tan (I) = 2 tan (lat)
# put it on the plot as a green dashed line
plt.axvline(gad,color='green',linestyle='dashed',linewidth=2)
plt.axvline(-gad,color='green',linestyle='dashed',linewidth=2)
plt.title('Inclinations')
pmagplotlib.label_tiepoints(ax2,100,ages_df.tiepoint.values,-1*ages_df.tiepoint_height.values,lines=True)
#

### "Christmas tree" of anisotropy
- use ipmag.ani\_depthplot()

In [ ]:
help(ipmag.ani_depthplot)

In [ ]:
ipmag.ani_depthplot(dir_path=dir_path)